# Part 3: CQL syntax - examples
### Goals
* To create more complex CQL queries
* To understand how to make some specific queries in Alma
    * Search by library
    * Search for inventory in institution
    * Special operators
    * CQL limitations
   
### How to use SRU
#### SRU servers
* https://swisscovery.slsp.ch/view/sru/41SLSP_NETWORK?version=1.2&operation=explain
* https://renouvaud.primo.exlibrisgroup.com/view/sru/41BCULAUSA_NETWORK?version=1.2&operation=explain
* https://www.sudoc.abes.fr/cbs/sru/?operation=explain&version=1.1

#### Documentation
* https://slsp.ch/fr/metadonnees/
* https://developers.exlibrisgroup.com/alma/integrations/sru/
* https://knowledge.exlibrisgroup.com/Alma/Product_Documentation/Alma_Online_Help_(Francais)/130Int%C3%A9grations_avec_des_syst%C3%A8mes_externes/030Gestion_des_ressources/170Recherche_SRU%2F%2FSRW
* https://abes.fr/wp-content/uploads/2023/05/guide-utilisation-service-sru-catalogue-sudoc.pdf

In [ ]:
import requests
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True, remove_comments=True, ns_clean=True)

ns = {
    "srw": "http://www.loc.gov/zing/srw/",
    "marc": "http://www.loc.gov/MARC21/slim"
}

## Search by library
To search directly in holdings of the libraries, we need to use an SRU server at the IZ level.

Code of the libraries cannot be used in SRU, we need the ID of the library. We can see it in html of Primo or in response of API responses about library information.

In [ ]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.holding_Library=112062990005504 and alma.title=Arbre',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_UBS'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

In [ ]:
for record in xml.findall('.//marc:record', namespaces=ns):
    print('MMS ID: ', record.findtext('marc:controlfield[@tag="001"]', namespaces=ns))
    for ava_f in record.findall('.//marc:datafield[@tag="AVA"]', namespaces=ns):
        print('\t', ava_f.findtext('marc:subfield[@code="q"]', namespaces=ns), ' - ', ava_f.findtext('marc:subfield[@code="e"]', namespaces=ns))

In [ ]:
print(etree.tostring(ava_f, pretty_print=True, encoding='unicode'))

## Search for inventory in institutions

We use the network SRU server. Holdings of the institutions are present in the 852 datafield.

In [ ]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.title=Arbre',
    'maximumRecords': '20'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

In [ ]:
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    izs = []
    for f852 in record.findall('.//marc:datafield[@tag="852"]', namespaces=ns):
        
        iz = f852.findtext('marc:subfield[@code="a"]', namespaces=ns)
        if iz is not None:
            izs.append(iz)
    if len(izs) > 0:
        print(f'{mms_id}: {", ".join(izs)}')
    else:
        print(f'{mms_id}: IN NETWORK ONLY')

In [ ]:
print(etree.tostring(f852, pretty_print=True, encoding='unicode'))

## Special syntax
* We need to use `==` when the item to search contains `-` or `_`, it means exact match
* We need to use `""` when item to search contains `<`, `>`, `=`, `/`, `(`, `)` and whitespace
* Indexes can cover more than one field

In [ ]:
# With "=" we consider "-" and "_" as whitespaces, all words need to match
# With "==" the entire string should match
# Be sure to check the results afterwards...

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios"', # 2 results
#    'query': 'alma.title="Les trios d\'anches de l\'Oiseau-lyre pour"', # 2 results
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre pour"', # 0 result
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre"', # 3 result due to 830a
    'query': 'alma.title="Les trios d\'anches de l\'Oiseau-lyre"', # 10 results
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    title = record.findtext('marc:datafield[@tag="245"]/marc:subfield[@code="a"]', namespaces=ns)
    parent_title = record.findtext('marc:datafield[@tag="830"]/marc:subfield[@code="a"]', namespaces=ns)
    print(mms_id, title, ' / ', parent_title)

In [ ]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.other_system_number="(swissbib)12675375*"',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    print(mms_id)

In [ ]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.other_system_number="(RERO)R009082738"',
    'maximumRecords': '1'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
print(r.text)

## CQL limitations
In Alma, "not" boolean is not supported. In some indexes we can use "<>" instead of "not".